# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察均值編碼的效果

# [作業重點]
- 仿造範例, 完成標籤編碼與均值編碼搭配邏輯斯迴歸的預測
- 觀察標籤編碼與均值編碼在特徵數量 / 邏輯斯迴歸分數 / 邏輯斯迴歸時間上, 分別有什麼影響 (In[3], Out[3], In[4], Out[4]) 

# 作業1
* 請仿照範例，將鐵達尼範例中的類別型特徵改用均值編碼實作一次

In [129]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

data_path = 'D:\\dataimport\\'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [130]:
df.dtypes

Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Cabin        object
Embarked     object
dtype: object

In [131]:
df.columns

Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')

In [132]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Object Features : {object_features}\n')

# 只留類別型欄位
df_obeject = df[object_features]
df_obeject = df_obeject.fillna('None')
train_num = train_Y.shape[0]
df_obeject.head()

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


In [133]:
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype != 'object':
        num_features.append(feature)
print(f'{len(object_features)} Number Features : {object_features}\n')


df_num = df[num_features]
df_num = df_num.fillna(df_num.median())
df_num.head()

5 Number Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


In [134]:
df_obeject.shape

(1309, 5)

In [135]:
df_num.shape

(1309, 5)

In [136]:
df_num.isnull().sum()

Pclass    0
Age       0
SibSp     0
Parch     0
Fare      0
dtype: int64

In [137]:
df_num.Age.unique()

array([22.  , 38.  , 26.  , 35.  , 28.  , 54.  ,  2.  , 27.  , 14.  ,
        4.  , 58.  , 20.  , 39.  , 55.  , 31.  , 34.  , 15.  ,  8.  ,
       19.  , 40.  , 66.  , 42.  , 21.  , 18.  ,  3.  ,  7.  , 49.  ,
       29.  , 65.  , 28.5 ,  5.  , 11.  , 45.  , 17.  , 32.  , 16.  ,
       25.  ,  0.83, 30.  , 33.  , 23.  , 24.  , 46.  , 59.  , 71.  ,
       37.  , 47.  , 14.5 , 70.5 , 32.5 , 12.  ,  9.  , 36.5 , 51.  ,
       55.5 , 40.5 , 44.  ,  1.  , 61.  , 56.  , 50.  , 36.  , 45.5 ,
       20.5 , 62.  , 41.  , 52.  , 63.  , 23.5 ,  0.92, 43.  , 60.  ,
       10.  , 64.  , 13.  , 48.  ,  0.75, 53.  , 57.  , 80.  , 70.  ,
       24.5 ,  6.  ,  0.67, 30.5 ,  0.42, 34.5 , 74.  , 22.5 , 18.5 ,
       67.  , 76.  , 26.5 , 60.5 , 11.5 ,  0.33,  0.17, 38.5 ])

# 作業2
* 觀察鐵達尼生存預測中，均值編碼與標籤編碼兩者比較，哪一個效果比較好? 可能的原因是什麼?

In [ ]:
### Label_means did the better at predicting, because the labeal can relative to the target values.
### I think name_mean is quite weird, the unique value of it is too much. It will lead to overfitting.

In [138]:
df_updated = pd.concat([df_obeject,df_num], axis=1)

In [139]:
df_updated.shape

(1309, 10)

In [140]:
df_updated   ### concatenate the object table and number table.

,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S,3,22.0,1,0,7.2500
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,1,38.0,1,0,71.2833
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S,3,26.0,0,0,7.9250
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,1,35.0,1,0,53.1000
4,"Allen, Mr. William Henry",male,373450,None,S,3,35.0,0,0,8.0500
5,"Moran, Mr. James",male,330877,None,Q,3,28.0,0,0,8.4583
6,"McCarthy, Mr. Timothy J",male,17463,E46,S,1,54.0,0,0,51.8625
7,"Palsson, Master. Gosta Leonard",male,349909,None,S,3,2.0,3,1,21.0750
8,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,347742,None,S,3,27.0,0,2,11.1333
9,"Nasser, Mrs. Nicholas (Adele Achem)",female,237736,None,C,2,14.0,1,0,30.0708


In [22]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸

In [141]:
df_temp = pd.DataFrame()
for c in df_obeject.columns:
    df_temp[c] = LabelEncoder().fit_transform(df_obeject[c])
train_X = df_temp[:train_num]
estimator = LogisticRegression()
start = time.time()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 5)
score : 0.780004837244799
time : 0.03007793426513672 sec


In [ ]:
# 對照組 : 標籤編碼 + number table + 邏輯斯迴歸 

In [142]:
df_temp = pd.DataFrame()
for c in df_obeject.columns:
    df_temp[c] = LabelEncoder().fit_transform(df_obeject[c])
    
train_X = pd.concat([df_temp[:train_num],df_num[:train_num]], axis=1)


estimator = LogisticRegression()
start = time.time()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 10)
score : 0.7879207188344367
time : 0.04913330078125 sec


In [156]:
# 均值編碼  + 邏輯斯迴歸

In [157]:
data = pd.concat([df_obeject[:train_num], train_Y], axis=1)
for c in df_obeject.columns:
    mean_df = data.groupby([c])['Survived'].mean().reset_index()
    mean_df.columns = [c, f'{c}_mean']
    data = pd.merge(data, mean_df, on=c, how='left')
    data = data.drop([c] , axis=1)
data = data.drop(['Survived'] , axis=1)    ### I think name_mean is quite weird, the unique value of it is too much.
#data = pd.concat([data,df_num[:train_num]], axis=1)

estimator = LogisticRegression()
start = time.time()
print(f'shape : {data.shape}')
print(f'score : {cross_val_score(estimator, data, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 5)
score : 1.0
time : 0.01704573631286621 sec


In [ ]:
# 均值編碼 + number table + 邏輯斯迴歸

In [154]:
data = pd.concat([df_obeject[:train_num], train_Y], axis=1)
for c in df_obeject.columns:
    mean_df = data.groupby([c])['Survived'].mean().reset_index()
    mean_df.columns = [c, f'{c}_mean']
    data = pd.merge(data, mean_df, on=c, how='left')
    data = data.drop([c] , axis=1)
data = data.drop(['Survived','Name_mean'] , axis=1)    ### I think name_mean is quite weird, the unique value of it is too much.
data = pd.concat([data,df_num[:train_num]], axis=1)

estimator = LogisticRegression()
start = time.time()
print(f'shape : {data.shape}')
print(f'score : {cross_val_score(estimator, data, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 9)
score : 0.977590647804249
time : 0.02213430404663086 sec


In [ ]:
### Label_means did the better at predicting, because the labeal can relative to the target values.
### I think name_mean is quite weird, the unique value of it is too much. It will lead to overfitting.